In [ ]:
import dspy
from src.roma_dspy.modules import Executor, Atomizer, Planner, Aggregator
from src.roma_dspy.engine import RecursiveSolver
from src.roma_dspy.engine.visualizer import ExecutionVisualizer

executor_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
atomizer_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
planner_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)
aggregator_lm = dspy.LM("openrouter/google/gemini-2.5-flash", cache=False)

# Initialize modules
atomizer = Atomizer(lm=atomizer_lm)
planner = Planner(lm=planner_lm)
executor = Executor(lm=executor_lm)
aggregator = Aggregator(lm=aggregator_lm)

# Create solver
solver = RecursiveSolver(
    atomizer,
    planner,
    executor,
    aggregator,
    max_depth=1
)

In [ ]:
# Test task - use a simpler task for clearer output
task_goal = "Write a haiku about programming"

print("=" * 80)
print("ROMA-DSPy Detailed Execution Visualization")
print("=" * 80)
print(f"\n🎯 Goal: {task_goal}\n")

# Execute task
print("⚙️  Executing task...\n")
result = solver.solve(task_goal)

In [ ]:
# Get the DAG for detailed visualization
dag = solver.last_dag

if dag:
    # Show the full execution report
    visualizer = ExecutionVisualizer()
    print(visualizer.get_full_execution_report(result, dag))

    # Also show the tree view with details
    print("\n" + "=" * 80)
    print("🌳 EXECUTION TREE WITH DETAILS")
    print("=" * 80)
    print(visualizer.get_execution_tree_with_details(result, dag))
else:
    print("No DAG available for visualization")